# Mainly for streamlit app
Download and store the movie posters automatically in your local drive

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pylab as plt
%matplotlib inline

df = pd.read_csv("../../Data/ml-latest-small/PreprocessedData_ml_latest_year_small.csv",index_col=0)
movies = df[['movieId','title', 'year']].drop_duplicates().sort_values(by=['movieId']).reset_index(drop=True)
#removing the year and paranthese from the tilte column
#movies["title"] = movies["title"].str.replace("(\(.*?\))", "").str.strip()
movies.head()

,movieId,title,year
0,1,Toy Story (1995),1995
1,2,Jumanji (1995),1995
2,3,Grumpier Old Men (1995),1995
3,4,Waiting to Exhale (1995),1995
4,5,Father of the Bride Part II (1995),1995


In [2]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9719 entries, 0 to 9718
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  9719 non-null   int64 
 1   title    9719 non-null   object
 2   year     9719 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 227.9+ KB


In [3]:
movies.head()

,movieId,title,year
0,1,Toy Story (1995),1995
1,2,Jumanji (1995),1995
2,3,Grumpier Old Men (1995),1995
3,4,Waiting to Exhale (1995),1995
4,5,Father of the Bride Part II (1995),1995


# movieposters: A simple Python package to get the link a movie's poster given its title.

The use is demonstrated below.

In [4]:
import movieposters as mp
link = mp.get_poster(title='Toy Story (1995)')

In [5]:
link

'https://m.media-amazon.com/images/M/MV5BMDU2ZWJlMjktMTRhMy00ZTA5LWEzNDgtYmNmZTEwZTViZWJkXkEyXkFqcGdeQXVyNDQ2OTk4MzI@._V1_QL75_UX380_CR0,2,380,562_.jpg'

> We have got the link for the poster.

In [6]:
import requests # request img from web
import shutil # save img locally


In [7]:
res = requests.get(link, stream = True)
# adding stream: True guarantees no interruptions will occur when the method is running.

In [8]:
FileName="../../MoviePosters/Toy Story (1995).png"

In [9]:
if res.status_code == 200:
    with open(FileName,'wb') as f:
        shutil.copyfileobj(res.raw, f)
    print('Image sucessfully Downloaded: ',FileName)
else:
    print('Image Couldn\'t be retrieved')

Image sucessfully Downloaded:  ../../MoviePosters/Toy Story (1995).png


# Now we will loop through all unique movies.

In [10]:
import os 

i=0
existing=0

failed_posters=[]
for movie in movies.title.unique():
    
    movieId=movies[movies.title==movie]['movieId'].unique()[0]
    movie_file="../../MoviePosters/"+str(movieId)+".png"
    
    #Since the loop is a big loop, only download posters not existing in the folder already
    if os.path.isfile(movie_file):
        existing += 1
        continue
    else:
        #Only download if the poster information exists
        try:
            movie_link = mp.get_poster(title=movie)
        except:
            failed_posters.append(movie)
            continue
        
        movie_res = requests.get(movie_link, stream = True)
    
        if res.status_code >= 200 or res.status_code<=229:
            with open(movie_file,'wb') as f:
                shutil.copyfileobj(movie_res.raw, f)
                i+=1
            #print('Image sucessfully Downloaded: ',movie_file)
        else:
            print('Image Couldn\'t be retrieved')
                
print('Sucessfully Downloaded: ',i,'posters')

Sucessfully Downloaded:  5074 posters


In [11]:
default_pic="../../MoviePosters/Default.png"

print("Posters for",len(failed_posters),"Movies not found.")

for nopic_movie in failed_posters:
    movieId=movies[movies.title==nopic_movie]['movieId'].unique()[0]
    movie_file="../../MoviePosters/"+str(movieId)+".png"
    shutil.copyfile(default_pic, movie_file)

Posters for 372 Movies not found.
